In [2]:
import pandas as pd
import numpy as np

In [3]:
class Node:
    def __init__(
        self,
        y: list,
        features: list,
        subset: pd.DataFrame,
        parent=None,
        stopping_depth=None,
        depth=None,
        feature=None,
    ):
        
        self.y = y
        self.subset = subset
        self.yclass = self.subset.columns[0]
        
        self.stopping_depth = stopping_depth
        self.depth = depth
        
        self.feature = feature
        self.features = features
        
        self.p = len(self.subset[self.subset[self.yclass]==y[0]])
        self.e = len(self.subset[self.subset[self.yclass]==y[1]])
        
        self.parent = parent
        self.children = []
        
        self.h = self.get_entropy(self.subset)
        self.best_feature = self.get_best_feature()
        
        
    def get_entropy(self, subset):
        
        yclass = self.yclass
        y = self.y
        
        s = len(subset)
        
        p = len(subset[subset[yclass]==y[0]])
        e = len(subset[subset[yclass]==y[1]])
            
        
        if s != 0 and p != 0 and e != 0:
            
            hx = -(p/s)*np.log2(p/s)-(e/s)*np.log2(e/s)
            
            return hx
        
        else:
            
            return 0
    
    
    def get_best_feature(self):
        
        sums=[]
        mainset = self.subset
        
        for feature in features:
            
            sum = 0
            
            for x in mainset[feature].unique():
                
                subset = mainset[mainset[feature]==x]
                hx = self.get_entropy(subset)
                sum += hx*(len(subset)/len(mainset))
                
            sums.append(self.h-sum)
        
        best_feature = features[sums.index(max(sums))]
        return best_feature
        
        
    def train(self):
        
        best_feature = self.best_feature
        mainset = self.subset.copy()
        
        for x in mainset[best_feature].unique():
            
            if len(self.features) != 0 and self.depth < self.stopping_depth:
                
                yclass = self.yclass
                features = self.features.copy()
                features.remove(best_feature)
                y = self.y
                subset = mainset[mainset[best_feature]==x]
                
                child_node = Node(self.y, features, subset, self, self.stopping_depth, self.depth+1, x)
                self.children.append(child_node)
                
                if len(subset[subset[yclass]==y[0]]) != 0 and len(subset[subset[yclass]==y[1]]) != 0:
                    
                    child_node.train()
        
        
    def print_content(self):
        
        const = self.depth * 4
        spaces = "-" * const
        
        if self.parent == None:
            print("Root")
        else:
            print(spaces + "|Split: " + str(self.parent.best_feature) + " : " + str(self.feature))
            print(' ' * const + "   | Entropy of the node: "+ str(round(self.h, 2)))
            print(' ' * const + "   | Y1 of the node: "+ str(round(self.p, 2)))
            print(' ' * const + "   | Y2 of the node: "+ str(round(self.e, 2)))
        
        
    def print_tree(self):
 
        self.print_content()
        
        if self.children != []:
            for node in self.children:
                node.print_tree()
        
    

    def predict(self, x, dataset):
        
        if self.children == []:
            
            if self.p >= self.e:
                return self.y[0]
            else:
                return self.y[1]
            
        else:
            
            for node in self.children:
                if node.feature == dataset[self.best_feature][x]:
                    
                    return node.predict(x, dataset)
   

    def test(self, dataset):
        
        predictions = [0, 0]
        
        for x in dataset.index:
            
            if self.predict(x, dataset) == dataset[self.yclass][x]:
                predictions[0] += 1
                
            else:
                predictions[1] += 1
        
        return predictions
        

In [4]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
pima = pd.read_csv(url, header=None) 
pima = pima[pima[11] != "?"]

features=list(pima.columns[1:])

X_train = pima.sample(frac = 0.7)
x_test = pima.drop(X_train.index)

root = Node(pima[pima.columns[0]].unique(), features, X_train, None, 3, 0, None)
root.train()
root.print_tree()

predictions = root.test(x_test)
accuracy = (predictions[0]/(predictions[0]+predictions[1])) *100
print("Accuracy: " + str(accuracy) + "%")

Root
----|Split: 5 : f
       | Entropy of the node: 0
       | Y1 of the node: 1100
       | Y2 of the node: 0
----|Split: 5 : n
       | Entropy of the node: 0.2
       | Y1 of the node: 62
       | Y2 of the node: 1890
--------|Split: 20 : k
           | Entropy of the node: 0
           | Y1 of the node: 0
           | Y2 of the node: 923
--------|Split: 20 : n
           | Entropy of the node: 0
           | Y1 of the node: 0
           | Y2 of the node: 904
--------|Split: 20 : r
           | Entropy of the node: 0
           | Y1 of the node: 51
           | Y2 of the node: 0
--------|Split: 20 : w
           | Entropy of the node: 0.61
           | Y1 of the node: 11
           | Y2 of the node: 63
------------|Split: 3 : p
               | Entropy of the node: 0
               | Y1 of the node: 0
               | Y2 of the node: 6
------------|Split: 3 : g
               | Entropy of the node: 0
               | Y1 of the node: 0
               | Y2 of the node: 5
------------